## Graphs: 

Denoted as ` G = (V, E) `
Problem statement -> node = `v`, feature = `x_v`, label = `t_v`
                  -> node = `h_v` [contains information on neighbourhood]
                  ->  `x_co[v]` = features of edges onneting with `v`
                  -> `h_ne[v]` = features of neighboring nodes of v
                  -> function `f` = transition function that projects inputs onto a d-dimensional space
                    - goal is to find unique solution of h_v 

```
v = [x_v, t_v]
h_v = f(x_v, x_co[v], h_ne[v], x_ne[v])
```

Banah fixed point theorem = **neighbourhood aggregation**
`H^t+1 = F(H^t, X)` -> H, X denotes matrix of all the h and x respectively [features of neighbours and all nodes]

Output of GNN computed by passing state h_v as well as feature x_v to an output function g -> 
` O_v = g(h_v, x_v) `

Deepwalk -> unsupervised node learning algorithm -> perform random walks on node to generate node sequenes, then run skip-gram to learn the embedding of each node based on the node sequqnees generated by random walks


In [3]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.model_selection import KFold, StratifiedKFold
from tqdm.auto import tqdm
import random
import os
from copy import deepcopy
import math
from glob import glob
from torch import nn
import functools
import shutil
from collections import OrderedDict
# from fastprogress import progress_bar

import torchvision.datasets as datasets
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import sklearn.metrics as metrics
import networkx
import torch_geometric
import torch_cluster, torch_sparse, torch_scatter
from torch_geometric.datasets import Planetoid

In [4]:
print(torch.__version__)
print(torch.version.cuda)

dataset = Planetoid(root="foldingoptimizer", name="Cora")

1.9.1
None


Processing...
Done!


In [15]:
print(dataset)
print(vars(dataset))
print("number of graphs: \t\t", len(dataset))
print("number of classes: \t\t", dataset.num_classes)
print("number of node features:\t", dataset.num_node_features)
print("number of edge features:\t", dataset.num_edge_features)



Cora()
{'name': 'Cora', 'root': 'foldingoptimizer', 'transform': None, 'pre_transform': None, 'pre_filter': None, '_indices': None, 'data': Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708]), 'slices': None, '_data_list': None, 'split': 'public'}
number of graphs: 		 1
number of classes: 		 7
number of node features:	 1433
number of edge features:	 0


In [16]:
print(dataset.data)

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [17]:
print("edge_index:\t\t", dataset.data.edge_index.shape)
print(dataset.data.edge_index)
print("\ntrain_mask:\t\t", dataset.data.train_mask.shape) #masks indicate node-feature = 2708, num_nodes = 1433
print(dataset.data.train_mask)
print("\nx:\t\t", dataset.data.x.shape)
print(dataset.data.x)
print("\ny:\t\t", dataset.data.y.shape)
print(dataset.data.y)

edge_index:		 torch.Size([2, 10556])
tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
        [ 633, 1862, 2582,  ...,  598, 1473, 2706]])

train_mask:		 torch.Size([2708])
tensor([ True,  True,  True,  ..., False, False, False])

x:		 torch.Size([2708, 1433])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

y:		 torch.Size([2708])
tensor([3, 4, 4,  ..., 3, 3, 3])


In [20]:
#Simple Example
from torch_geometric.data import Data

edge_index = torch.tensor([ [0,1,1,2], [1,0,2,1] ], dtype=torch.long)
#0->1, 1->0, 1->2, 2->1 [e1_i -> e2_i]
#this specifies direction, so undirected nodes need 2 copies

x = torch.tensor([[-1], [0], [1]], dtype=torch.float)
data = Data(x=x, edge_index=edge_index)
print(data)
print(Data(edge_index=[2, 4], x = [3, 1]))

Data(x=[3, 1], edge_index=[2, 4])
Data(x=[2], edge_index=[2])


![](https://pytorch-geometric.readthedocs.io/en/latest/_images/graph.svg)

In [ ]:
import os.path as ops
